# Random Forest Explainability Analysis

Importing necessary packages:

In [7]:
import os
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import ShuffleSplit
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
import graphviz
import seaborn as sns
import warnings

warnings.filterwarnings("ignore")

UTILS:

In [8]:
DATABASE_DIR = '/home/macdowell/Workspace/Explainability-Model-Analysis/database/'
RESULTS_DIR = 'results/'
IMAGES_DIR = 'images/'
MODEL_NAME = 'RandomForest/'

def get_entries_name_inside_dir(path):
    smell_folders = os.listdir(path)
    return smell_folders

def dataframe_appended_from_smell_folder(folder_path):
    csv_files = os.listdir(folder_path)
    df = None
    
    for csv in csv_files:
        if 'lock' in csv:
            continue
        
        if df is None:
            df = pd.read_csv(folder_path + '/' + csv)
        else:
            df_aux = pd.read_csv(folder_path + '/' + csv)
            df = pd.concat([df, df_aux])
    return df

def dict_to_csv(output_path, dict_to_save, column_name):
    
    df = pd.DataFrame.from_dict(dict_to_save, orient='columns')
    df.index.names = [column_name]
    df.to_csv(output_path + '.csv')

### Effectivity on differents database's percents:

- RQ1)

In [9]:
def evaluate_model(X, y, df, train_percent, effectivity_dict, smell, cv_splits):
    
    ss = ShuffleSplit(n_splits=cv_splits, train_size=train_percent, test_size=0.25)
    score_list = []
    
    for train_index, test_index in ss.split(X, y):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        
        score = perform_models(
            [
                RandomForestClassifier()
            ],
            X_train, X_test,
            y_train, y_test,
            df,
            train_percent,
            effectivity_dict,
            smell
        )
        
        score_list.append(score)
        
        # Adding scores into dict:
    effectivity_dict[smell][train_percent] = sum(score_list)/len(score_list)
        

def perform_models(classifiers, X_train, X_test, y_train, y_test, df, train_percent, effectivity_dict, smell):

    for classifier in classifiers:
        # Creating key index in dict to save evaluation metrics value:
        #string += classifier.__class__.__name__

        # Train:
        classifier.fit(X_train, y_train)
        
        #TODO: Make Viz woks
        # Viz:
        '''
        dot_data = tree.export_graphviz(classifier, out_file=None, 
                      feature_names=df.columns,  
                      class_names=['smell', 'not smell'],
                      filled=True, rounded=True,  
                      special_characters=True)  
        graph = graphviz.Source(dot_data)  
        graph
        '''
        
        # Predicting values with model:
        predicteds = classifier.predict(X_test)
        
        # Getting score metrics:
        return f1_score(y_test, predicteds)
        

smell_folders = get_entries_name_inside_dir(DATABASE_DIR)
train_percents = [0.25, 0.5, 0.75]

effectivity_dict = {}

for smell in smell_folders:
    smell_metrics_df = dataframe_appended_from_smell_folder(DATABASE_DIR + smell)

    effectivity_dict[smell] = {}
    
    for percent in train_percents:
        
        effectivity_dict[smell][percent] = -1
        
        if '0' in smell_metrics_df.columns:
            smell_metrics_df = smell_metrics_df.drop(columns=['0'], axis=1)

        for c in smell_metrics_df.columns:
            smell_metrics_df[c] = np.nan_to_num(smell_metrics_df[c])

        X = smell_metrics_df.iloc[:, smell_metrics_df.columns != 'Smell']
        y = smell_metrics_df.iloc[:, smell_metrics_df.columns == 'Smell']
        
        evaluate_model(X, y, smell_metrics_df, percent, effectivity_dict, smell, cv_splits=5)

#print(effectivity_dict)
# Writing results in CSV
if not os.path.isdir(RESULTS_DIR + MODEL_NAME):
    os.mkdir(RESULTS_DIR + MODEL_NAME)
dict_to_csv(RESULTS_DIR + MODEL_NAME + 'train_percent', effectivity_dict, 'train_percent')

### PLOT ABOUT RQ1)

In [10]:
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt
#Size of the plot
plt.rcParams["figure.figsize"] = [9,6]
df = pd.read_csv(RESULTS_DIR + MODEL_NAME + 'train_percent.csv')

for smell in smell_folders:
    ax = sns.lineplot(x="train_percent", y=smell, data=df)
    ax.set(xlabel='Database train percent', ylabel='f1 score')
    fig = ax.get_figure()

    if not os.path.isdir(IMAGES_DIR + MODEL_NAME):
        os.mkdir(IMAGES_DIR + MODEL_NAME)
        os.mkdir(IMAGES_DIR + MODEL_NAME + 'train_percent/')

    fig.savefig(IMAGES_DIR + MODEL_NAME + 'train_percent/' + smell + ".png")
    fig.clf()

<Figure size 648x432 with 0 Axes>

### The maximum depth of the tree:

- RQ2)

Tune parameter `max_depth` = (3, 5, 7, 10)

In [12]:
def evaluate_model(X, y, df, max_depth, effectivity_dict, smell, cv_splits):
    
    ss = ShuffleSplit(n_splits=cv_splits, train_size=0.70, test_size=0.30)
    score_list = []
    
    for train_index, test_index in ss.split(X, y):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        
        score = perform_models(
            [
                RandomForestClassifier(max_depth=max_depth)
            ],
            X_train, X_test,
            y_train, y_test,
            df,
            effectivity_dict,
            smell
        )
        
        score_list.append(score)
        
        # Adding scores into dict:
    effectivity_dict[smell][max_depth] = sum(score_list)/len(score_list)
        

def perform_models(classifiers, X_train, X_test, y_train, y_test, df, effectivity_dict, smell):

    for classifier in classifiers:
        # Creating key index in dict to save evaluation metrics value:
        #string += classifier.__class__.__name__

        # Train:
        classifier.fit(X_train, y_train)
        
        #TODO: Make Viz woks
        # Viz:
        '''
        dot_data = tree.export_graphviz(classifier, out_file=None, 
                      feature_names=df.columns,  
                      class_names=['smell', 'not smell'],
                      filled=True, rounded=True,  
                      special_characters=True)  
        graph = graphviz.Source(dot_data)  
        graph
        '''
        
        # Predicting values with model:
        predicteds = classifier.predict(X_test)
        
        # Getting score metrics:
        return f1_score(y_test, predicteds)


smell_folders = get_entries_name_inside_dir(DATABASE_DIR)
tree_max_depth = [3, 5, 7, 10]

effectivity_dict = {}

for smell in smell_folders:
    smell_metrics_df = dataframe_appended_from_smell_folder(DATABASE_DIR + smell)

    effectivity_dict[smell] = {}
    
    for max_depth in tree_max_depth:
        
        effectivity_dict[smell][max_depth] = -1.0
        
        if '0' in smell_metrics_df.columns:
            smell_metrics_df = smell_metrics_df.drop(columns=['0'], axis=1)

        for c in smell_metrics_df.columns:
            smell_metrics_df[c] = np.nan_to_num(smell_metrics_df[c])

        X = smell_metrics_df.iloc[:, smell_metrics_df.columns != 'Smell']
        y = smell_metrics_df.iloc[:, smell_metrics_df.columns == 'Smell']
        
        evaluate_model(X, y, smell_metrics_df, max_depth, effectivity_dict, smell, cv_splits=5)
    

if not os.path.isdir(RESULTS_DIR + MODEL_NAME):
    os.mkdir(RESULTS_DIR + MODEL_NAME)
dict_to_csv(RESULTS_DIR + MODEL_NAME + 'max_depth', effectivity_dict, 'max_depth')

### Plot about RQ2)

In [13]:
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt
#Size of the plot
plt.rcParams["figure.figsize"] = [9,6]
df = pd.read_csv(RESULTS_DIR + MODEL_NAME + 'max_depth.csv')

for smell in smell_folders:
    ax = sns.lineplot(x="max_depth", y=smell, data=df)
    ax.set(xlabel='Max depth of tree', ylabel='f1 score')
    fig = ax.get_figure()
    
    if not os.path.isdir(IMAGES_DIR + MODEL_NAME + 'max_depth/'):
        os.mkdir(IMAGES_DIR + MODEL_NAME + 'max_depth/')

    fig.savefig(IMAGES_DIR + MODEL_NAME + 'max_depth/' + smell + ".png")
    fig.clf()

<Figure size 648x432 with 0 Axes>

### Effectivity on different database's percents with different maximum depth of the tree:
- RQ3)

Train with 25%, 50%, 75% and max depth of 3, 5, 7, 10..

In [14]:
def evaluate_model(X, y, df, train_percent, max_depth, effectivity_dict, smell, cv_splits):
    
    ss = ShuffleSplit(n_splits=cv_splits, train_size=train_percent, test_size=0.25)
    score_list = []
    
    for train_index, test_index in ss.split(X, y):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        
        score = perform_models(
            [
                RandomForestClassifier(max_depth=max_depth)
            ],
            X_train, X_test,
            y_train, y_test,
            df,
            effectivity_dict
        )
        
        score_list.append(score)
        
        # Adding scores into dict:
    effectivity_dict[smell][train_percent][max_depth] = sum(score_list)/len(score_list)
        

def perform_models(classifiers, X_train, X_test, y_train, y_test, df, effectivity_dict):

    for classifier in classifiers:
        # Creating key index in dict to save evaluation metrics value:
        #string += classifier.__class__.__name__

        # Train:
        classifier.fit(X_train, y_train)
        
        #TODO: Make Viz woks
        # Viz:
        '''
        dot_data = tree.export_graphviz(classifier, out_file=None, 
                      feature_names=df.columns,  
                      class_names=['smell', 'not smell'],
                      filled=True, rounded=True,  
                      special_characters=True)  
        graph = graphviz.Source(dot_data)  
        graph
        '''
        
        # Predicting values with model:
        predicteds = classifier.predict(X_test)
        
        # Getting score metrics:
        return f1_score(y_test, predicteds)
        

smell_folders = get_entries_name_inside_dir(DATABASE_DIR)

train_percents = [0.25, 0.5, 0.75]
tree_max_depth = [3, 5, 7, 10]

effectivity_dict = {}

for smell in smell_folders:
    smell_metrics_df = dataframe_appended_from_smell_folder(DATABASE_DIR + smell)

    effectivity_dict[smell] = {}
    
    for percent in train_percents:
        
        effectivity_dict[smell][percent] = {}
        
        for max_depth in tree_max_depth:

            effectivity_dict[smell][percent][max_depth] = -1
            
            if '0' in smell_metrics_df.columns:
                smell_metrics_df = smell_metrics_df.drop(columns=['0'], axis=1)

            for c in smell_metrics_df.columns:
                smell_metrics_df[c] = np.nan_to_num(smell_metrics_df[c])

            X = smell_metrics_df.iloc[:, smell_metrics_df.columns != 'Smell']
            y = smell_metrics_df.iloc[:, smell_metrics_df.columns == 'Smell']

            evaluate_model(X, y, smell_metrics_df, percent, max_depth, effectivity_dict, smell, cv_splits=5)
            
if not os.path.isdir(RESULTS_DIR + MODEL_NAME):
    os.mkdir(RESULTS_DIR + MODEL_NAME)
dict_to_csv(RESULTS_DIR + MODEL_NAME + 'train_percent_max_depth', effectivity_dict, 'train_percent')